<a href="https://colab.research.google.com/github/ltittle/QNC/blob/main/Confidence_Intervals_and_Bootstrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definition

A confidence interval quantifies a range of potential values that a parameter of a population (e.g., its mean value) can take, based on sampled observations (i.e., the data you have). In other words, the confidence interval is a measure of uncertainty: the wider the interval, the more uncertainty there is on the actual value of the parameter, given the data. Confidence intervals are a foundational piece of all rigorous, quantitative analyses of data and in fact have been proposed to be one of the three pillars of "[the new statistics](https://journals.sagepub.com/doi/full/10.1177/0956797613504966)" (the other two being measures of effect sizes and the use of meta-analyses).

Exactly how confidence intervals should be interpreted is a matter of [debate](https://stats.stackexchange.com/questions/2272/whats-the-difference-between-a-confidence-interval-and-a-credible-interval) and depends (of course!) on whether it was computed using frequentist or Bayesian methods. In either case, they are computed at a pre-specified confidence level, which is typically 95% but can be other values (e.g., 90% or 99%).

Using Bayesian methods, the interval (typically referred to as a "credible interval") is interpreted in a more intuitive manner: there is a 95% (or whatever value that is specified by the confidence level) chance that the true value is within the given interval (more specifically, 95% of the mass of the posterior probability distribution is within the given interval).

In contrast, a frequentist confidence interval implies that if we repeated the same experiment N times, the true value will fall within that interval 95% of the time.

The good news is that under a number of common conditions, the two approaches give very similar answers and so as a practical matter it might not matter very much which you use (e.g., [this paper](https://canvas.upenn.edu/courses/1358934/files/folder/Courses/Quantitative%20Neuro%20Core/Readings/Confidence%20Intervals%20and%20Bootstrapping/Confidence%20Intervals?preview=79166068)). The bad news is that there are conditions in which the two approaches do not give the same answers (see [here](https://pubmed.ncbi.nlm.nih.gov/26450628/) and [here](https://psyarxiv.com/we9h6)), which are worth being aware of as you compute confidence intervals.

# Four ways to compute a confidence (or credible) interval on an estimate of the mean of a population

Here we assume that we have *n* measurements ($x_i$) from a population that we know is [Gaussian distributed](https://github.com/PennNGG/Quantitative-Neuroscience/blob/master/Probability%20Distributions/Python/Gaussian%20(Normal).ipynb) (i.e., with a true mean $\mu$ and standard deviation $\sigma$). Thus the sample mean ($\bar{x}$) is computed as:

$\bar{x}=\frac{\Sigma{x_i}}{n}$.

We want to find a confidence interval on our estimate of the mean. This is a very common problem, and the different approaches described below will all give roughly the same answer.

## 1. The simple, analytic approach with large *n* and/or known standard deviation.

Here we compute the confidence interval in terms of multiples of the standard error of the mean, which is the standard deviation of the sampling distribution of the mean -- that is, the standard deviation of the distribution you would get if you repeated your experiment over and over and computed the sample mean each time.

This approach requires understanding several concepts:

a. Because of the [Central Limit Theorem](https://online.stat.psu.edu/stat414/book/export/html/750), the sampling distribution of the mean approaches a [normal (Gaussian)](https://github.com/PennNGG/Quantitative-Neuroscience/blob/master/Probability%20Distributions/Python/Gaussian%20(Normal).ipynb) distribution with large *n*; see [here](https://stats.stackexchange.com/questions/371067/trouble-relating-the-central-limit-theorem-to-confidence-intervals) and [here](https://online.stat.psu.edu/stat506/lesson/1/1.4) for discussions of how this idea relates to confidence intervals.

b. Accordingly, we assess the variability of the sampling distribution of the mean based on its (known) standard deviation. This value is known as the standard error of the mean (SEM), which can be computed from the true standard deviation (*s*) of the sampled distribution as:

$SEM=\frac{s}{\sqrt{n}}=\sqrt{\frac{\Sigma{(x-\bar{x})^2}}{n}}$

Careful: don't [confuse *s* and the SEM](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3148365/)!

c. The SEM thus indicates the range of values that are within one standard deviation of the mean value of the sampling distribution of the mean, which corresponds to ~68% of all possible values:

![](https://drive.google.com/uc?export=view&id=1hct1RNgwYziStIetfxlUNy12bx3qpXCS)

In other words, the notation $[\bar{x}-SEM, \bar{x}+SEM]$ indicates the lower and upper bounds of the 68% confidence interval on the mean.

d. The confidence interval for an arbitrary confidence level can be computed by simply multiplying the SEM by a factor that corresponds to the appropriate range of the sampling distribution of the mean. This factor is known as the [z-score](https://www.statisticshowto.com/probability-and-statistics/z-score/) and represents the number of standard deviations that a value is from the mean of a distribution. It can be determined using a [z-score calculator](http://www.z-table.com), which relates the z-score to the area under the curve that is to the left of that z-score (i.e., the total probability that the value is less than that z-score):

![](https://drive.google.com/uc?export=view&id=1MZVC7ez6N9w8MInWdzJl635q5L-IHf0z)

For a standard, two-sided confidence interval, the appropriate z-score is determined based on one-half of one minus the confidence level, because you want to find the area between two symmetric tails.

Let's say you want the 95% confidence interval (i.e., 0.95). Thus, you need to identify the z-score corresponding to an area equal to 0.5*(1–0.95) = 0.025, which is 1.96. Thus, the 95% confidence interval = $[\bar{x}-SEM\times 1.96, \bar{x}+SEM\times 1.96]$

You can get the z-score corresponding to a particular area under the curve to the left of that z-score in Matlab using [norminv](https://www.mathworks.com/help/stats/norminv.html) and in Python using the ppf method of Scipy's [norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html).



## 2. The simple, analytic approach with small *n* and unknown population standard deviation

When the true population standard deviation is not known (as is typically the case), the SEM must be calculated using the sample standard deviation (i.e., the standard deviation estimated from the data). As detailed [here](https://colab.research.google.com/drive/1Q_Du5NK71Rc1qu-byh8dy8Fs39uvR_4n?usp=sharing), the sample standard deviation is biased unless you use [Bessel's correction](https://www.statisticshowto.com/bessels-correction/) to minimize the bias. Using this correction, the sample distribution of the mean value of a normally distributed random variable follows a [Student's t-distribution](https://colab.research.google.com/drive/1Q_Du5NK71Rc1qu-byh8dy8Fs39uvR_4n?usp=sharing) that deviates most strongly from a normal distribution when *n* is relatively small (it has heavier tails than a normal distribution, which implies that there are more extreme values). Here computing confidence intervals is the same as above, but instead of using a z-score calculator (which assumes a normal distribution), you need to use a [t-distribution calculator](https://www.dummies.com/education/math/statistics/using-the-t-distribution-to-calculate-confidence-intervals/).

## 3. Bootstrapped confidence intervals

[Bootstrapping](https://projecteuclid.org/journals/annals-of-statistics/volume-7/issue-1/Bootstrap-Methods-Another-Look-at-the-Jackknife/10.1214/aos/1176344552.full) [available on [Canvas](https://canvas.upenn.edu/courses/1358934/files/folder/Courses/Quantitative%20Neuro%20Core/Readings/Confidence%20Intervals%20and%20Bootstrapping/Bootstrapping?preview=98867712)] is a powerful approach for estimating a sample distribution given a set of observed data. The idea is to resample (with replacement) from those data, each time computing the statistic of interest (e.g., the mean). You can then use this distribution directly to determine the confidence intervals.

Advantages of this approach are that it: 1) does not require assumptions about the nature of the population distribution; 2) can be applied equally well to multiple statistics (e.g., mean or median); and 3) is easy to compute.

Bootstrapping is based on the following logic (taken from this [nice description](https://ocw.mit.edu/courses/mathematics/18-05-introduction-to-probability-and-statistics-spring-2014/readings/MIT18_05S14_Reading24.pdf)):

1. $x_1, x_2,...,x_n$ is a data sample drawn from a distribution $F$.
2. $\mu$ is a statistic computed from the sample.
3. $F^∗$ is the empirical distribution of the data (the resampling distribution).
4. $x_1^*, x_2^*,...,x_n^*$
 is a resample of the data *of the same size as the original sample*.
5. $\mu^*$ is the statistic computed from the resample.

The idea is that you now have a distribution $F^*$ that can be used as a surrogate for $F$; that is, confidence intervals on $\mu$ are computed as confidence intervals on $\mu^*$.



## 4. Bayesian credible intervals

Credible intervals are a measure of our belief in a parameter of the population, given our data. We therefore compute the posterior probability distribution of the population mean $\mu$ given data samples $X={x_1, x_2,...,x_n}$ according to Bayes' Rule:

$p(\mu|X)=\frac{p(X|\mu)p(\mu)}{p(X)}$

Here $p(X)$ represents the probability of getting the data and is really just a normalizing constant (to make sure the area under the resulting probability distribution equals one), so we'll call it *k*. $p(\mu)$ is the prior on the mean; that is, what is the probability that the mean is equal to each particular value, determined before you see any data? We will assume this probability is uniform and thus will also refer to it as a constant, say *p*. That leaves just the likelihood term $p(X|\mu)$, which based on our assumptions above is a Gaussian (here let us assume that we know the standard deviation $\sigma$). Note that *X* is a set of *n* independent observations, so the total likelihood is the product of the likelihoods determined for each observation separately; that is:

$p(X|\mu)=\prod^n_{i-1}{\frac{1}{\sqrt{2\pi \sigma^2}}exp\left [ \frac{-(x_i-\mu)^2}{2\sigma^2} \right ]}$

Which we can rearrange and multiply by $p(\mu)=p$ and divide by $p(X)=k$ to get the posterior:

$p(\mu|X)=\frac{p}{k\sqrt{2\pi \sigma^2}}exp{\left [\frac{1}{2\sigma^2} \sum^n_{i-1}(x_i-\mu)^2 \right ]}$

Then using the definition of the sample mean ($\bar{x}=\frac{1}{n}\sum{x_i}$), [this shortcut for computing the variance](https://www.thoughtco.com/sum-of-squares-formula-shortcut-3126266), and rearranging some more gives:

$p(\mu|X)=\frac{p}{k\sqrt{2\pi \sigma^2}}exp{\left [\frac{1}{2\sigma^2} \left [ n^2\sigma^2+n\bar{x^2}-2n\mu\bar{x}^2 +n\mu^2\right ] \right ]}$

$p(\mu|X)=\frac{p}{k\sqrt{2\pi \sigma^2}}exp{\left [ -\frac{n^2\sigma^2}{s\sigma^2} -\frac{n}{2\sigma^2}(\mu-\bar{x})^2\right ]}$

Note that we rearranged terms to make it clear that we are considering different values of $\mu$ as they relate to the given sample mean $\bar{x}$. Let's rearrange one more time:

$p(\mu|X)=\frac{p}{k\sqrt{2\pi \sigma^2}}exp{\left [ -\frac{n^2\sigma^2}{s\sigma^2}\right ]}exp{\left[ -\frac{1}{2\frac{\sigma^2}{n}}(\mu-\bar{x})^2\right ]}$

This form shows that the posterior distribution for given X is itself a Gaussian with a mean value of $\bar{x}$ and a standard deviation of $\frac{\sigma}{\sqrt{n}}$; in other words, a distribution defined by the sample mean and the standard error of the mean! In this case, the credible interval is computed in exactly the same way as the confidence interval computed analytically from the sample mean and standard error of the mean described above.

# Exercises

Compute confidence/credible intervals based on the four methods above for simulated data sampled from a population that is Gaussian distributed with mean $\mu$=10 and standard deviation $\sigma$=2, for *n*=5, 10, 20, 40, 80, 160, 1000 at a 95% confidence level.

The answers will be found [here](https://github.com/PennNGG/Quantitative-Neuroscience/tree/master/Answers%20to%20Exercises/Python) after the due date.

In [10]:
import numpy as np

# Set random seed so results are reproducible
np.random.seed(42)

mu = 10
sigma = 2
ns = [5, 10, 20, 40, 80, 160, 1000]  # sample sizes

for n in ns:
    # simulate data
    data = np.random.normal(loc=mu, scale=sigma, size=n)
#Method 1:

    # compute mean and SEM
    xbar = data.mean()
    sem = data.std(ddof=1) / np.sqrt(n)  # sample SEM

    # simple interval [xbar - SEM, xbar + SEM]
    ci_lo = xbar - sem
    ci_hi = xbar + sem

    print(f"n={n:4d}, mean={xbar:.3f}, SEM={sem:.3f}, interval=({ci_lo:.3f}, {ci_hi:.3f})")


n=   5, mean=10.918, SEM=0.633, interval=(10.285, 11.551)
n=  10, mean=9.572, SEM=0.677, interval=(8.895, 10.249)
n=  20, mean=9.520, SEM=0.399, interval=(9.121, 9.920)
n=  40, mean=9.801, SEM=0.314, interval=(9.487, 10.115)
n=  80, mean=9.918, SEM=0.203, interval=(9.715, 10.121)
n= 160, mean=10.190, SEM=0.160, interval=(10.030, 10.349)
n=1000, mean=10.101, SEM=0.063, interval=(10.039, 10.164)


In [12]:
#Method 2
import numpy as np
from scipy.stats import t  # <-- add this

# example data
data = np.array([1, 2, 3, 4, 5])
n = len(data)
xbar = data.mean()
s = data.std(ddof=1)          # sample SD with Bessel's correction
sem = s / np.sqrt(n)          # standard error of the mean

t_crit = t.ppf(0.975, df=n-1) # 95% CI t-value

ci_lo = xbar - t_crit * sem
ci_hi = xbar + t_crit * sem

print(ci_lo, ci_hi)


1.036756838522439 4.9632431614775605


In [14]:
#Method 4: Bayesian credible intervals
import numpy as np

# example data
data = np.array([1, 2, 3, 4, 5])
n = len(data)
xbar = data.mean()
s = data.std(ddof=1)          # sample SD

# posterior of the mean (assuming noninformative prior)
# t-distribution approximation (like frequentist) or simulate posterior
# Here we'll simulate posterior using normal approximation
posterior_samples = np.random.normal(loc=xbar, scale=s/np.sqrt(n), size=10000)

# 95% credible interval
ci_lower = np.percentile(posterior_samples, 2.5)
ci_upper = np.percentile(posterior_samples, 97.5)

print(f"95% Bayesian credible interval: [{ci_lower:.3f}, {ci_upper:.3f}]")


95% Bayesian credible interval: [1.612, 4.385]


# Credits

Copyright 2021 by Joshua I. Gold, University of Pennsylvania

In [13]:
#Method 3: bootstrapping
import numpy as np

# example data
data = np.array([1, 2, 3, 4, 5])
n_boot = 10000  # number of bootstrap samples
alpha = 0.05    # for 95% CI

# generate bootstrap samples and compute their means
boot_means = np.array([np.mean(np.random.choice(data, size=len(data), replace=True))
                       for _ in range(n_boot)])

# compute CI
ci_lower = np.percentile(boot_means, 100 * (alpha/2))
ci_upper = np.percentile(boot_means, 100 * (1 - alpha/2))

print(f"95% bootstrap CI: [{ci_lower:.3f}, {ci_upper:.3f}]")


95% bootstrap CI: [1.800, 4.200]


In [ ]:
import scipy.stats